# Cvičenie 8: Práca so spojitými priestormi

Na predošlých cvičeniach sme videli, ako fungujú rôzne algoritmy učenia posilňovaním. Ako príklad prostredia sme použili jednoduchý dvojdimenzionálny gridworld *3x3* s deviatimi stavmi, kde stav prostredia bol určený pozíciou agenta. Aj keď dané prostredie nám dalo potrebnú intuíciu o fungovaní vybraných algoritmov RL, reálne problémy sú zvyčajne zložitejšie a prostredia v nich môžu mať omnoho viac (aj nekonečne veľa) možných stavov. Na dnešnom cvičení sa pozrieme na to, ako dokážeme pracovať s takýmito prostrediami.

## Prostredie Continuous Gridworld

Ako prostredie použijeme spojitú verziu gridworldu, ktorú sme implementovali minulý týždeň na cvičení. Toto prostredie vyjadruje identickú úlohu ako diskrétny gridworld, t.j. agent musí nájsť najkratšiu cestu k cieľovej pozícii bez toho, aby medzitým spadol do pasce. Rozdielom je, že namiesto toho, aby sa agent pohyboval medzi možnými diskrétnymi stavmi, jeho poloha bude popísaná dvomi hodnotami: pozícia po súradnici *y* a *x*, pričom obe tieto hodnoty môžu nadobúdať hodnotu z intervalu $<0, height>$, resp. $<0, width>$, kde *height* a *width* je výška a šírka sveta.

[Implementáciu spojitého gridworldu nájdete na tomto odkaze.](sources/lab08/gridworld.py) Preštudujte a zopakujte si implementačné detaily tohto kódu. Sústreďte sa na jednotlivé metódy a ich funkcionalitu.

## Diskretizácia

Vzhľadom na to, že teraz pracujeme so spojitým stavovým priestorom (t.j. prostredie má nekonečne veľa možných stavov), nevieme priamo použiť metódy založené na inkrementálnej aktualizácii odhadov hodnotých funkcií stavov, resp. akcií. K tomu, aby sme tak mohli urobiť (a použiť metódy ako Q-learning, SARSA, Monte Carlo algoritmy, atď.), potrebujeme náš spojitý priestor predspracovať, teda diskretizovať.

Cieľom diskretizácie je zo spojitého priestoru vytvoriť diskrétny priestor, v ktorom prostredie môže mať konečný počet stavov. Samozrejme existuje mnoho spôsobov diskretizácie, a vhodnosť ich použitia závisí od riešeného problému. Najjednoduchším spôsobom je ale namapovať segment spojitého priestoru do jedného diskrétneho stavu. Príkladom takejto diskretizácie je rozdeliť spojitý priestor na *n* segmentov, a predpokladať, že daný segment reprezentuje jeden diskrétny stav - teda všetky reálne stavy prostredia v danom segmente budeme považovať za príklad toho istého diskrétneho stavu. Samozrejme predpokladom na správnosť takéhoto riešenia je, že od agenta sa očakáva rovnaká funkcionalita v každom stave v rámci tohto segmentu, keďže politika agenta bude vytvorená na základe diskrétneho stavu.

Otázkou ostáva, ako rozdeliť stavový priestor na segmenty. Niekoľko základných prístupov:
* segmenty rovnakej veľkosti - všetky rozmery stavového priestoru rozdelíme na intervaly rovnakej veľkosti; diskrétny priestor bude popísaný niekoľkými hodnotami, ktoré nám budú určovať, v ktorom sme intervale po danej osi.
* segmenty na základe proporcionálneho vzorkovania - segmenty nebudú mať rovnaké veľkosti, ale intervaly budú menšie okolo často navštívených stavov (jemnejšia granularita) a väčšia v tých častiach stavového priestoru, ktoré agent navštívi menej často (hrubšia granularita).
* náhodné segmenty - v stavovom priestore vytvoríme niekoľko náhodných segmentov, ktoré sa môžu ale nemusia pretínať; konkrétne spojité stavy potom reprezentujeme vektorom, kde jednotlivé hodnoty vyjadrujú, či sa daný stav nachádza v jednotlivých segmentoch.

## Diskretizácia gridworldu

V tomto kroku vyskúšame jednoduchú diskretizáciu pre riešenie problému spojitého gridworldu pomocou Q-learningu. [Na tomto odkaze nájdete naívnu implementáciu Q-learningu](sources/lab08/qlearning.py), ktorá ale nedokáže pracovať so spojitým gridworldom, keďže Q-tabuľka očakáva celé čísla ako indexy, pozícia agenta je ale popísaná desatinnými číslami. Upravte kód tak, aby sa Q-learning naučil (skoro) optimálnu politiku navigácie v gridworldu.

Pri riešení postupujte nasledovne:
1. navrhnite spôsob diskretizácie
2. upravte rozmery Q-tabuľky ak je to potrebné
3. v triede `QLearning` implementujte metódu pre predspracovanie (diskretizáciu) stavu prostredia
4. upravte trénovanie tak, aby používalo metódu pre diskretizáciu stavu prostredia

Vaše riešenie vyskúšajte s rôznymi granularitami diskretizácie a spozorujte, ako sa mení politika agenta. Ak diskretizácia bude úspešná, agent by mal rozpoznať diskrétne stavy na pozícii cieľa ([3, 3]) a pasce ([2, 2]) ako terminálne.

## Práca so spojitým priestorom pomocou Q-sietí

Na 6. cvičení sme videli, ako fungujú Q-siete, ktoré by mali byť schopné priamo pracovať so spojitým priestorom, vzhľadom na to, že na vstupe majú priamo vektorovú reprezentáciu stavu prostredia. [Na tomto odkaze nájdete implementáciu Q-siete](sources/lab08/q-network.py), ktorá pracuje s naším spojitým gridworldom. Otestujte a vyhodnoťte trénovanie Q-siete.

## Spojitý priestor akcií

V predošlých krokoch sme videli, ako dokážeme pracovať so spojitými stavovými priestormi. Pre podporu spojitého priestoru akcií potrebujeme použiť trocha iný prístup, a to algoritmy typu *actor-critic*. V týchto algoritmoch používame dva typy agentov: jeden je actor, teda agent, ktorý vyberá akcie; druhý agent je critic, ktorého cieľom je vyhodnotiť správnosť a vhodnosť akcií vybratých actorom. V závislosti od implementácie títo agenti môžu byť implementovaní pomocou cieľových sietí, čo reálne znamená prácu so štyrmi sieťami.

Príkladom algoritmu actor-critic s podporou spojitého priestoru akcií je `DDPG`, teda *deep deterministic policy gradient*. Vstupom pre sieť actora je reprezentácia stavu, a výstupom je hodnota, ktorá určuje akciu. Napríklad, ak máme úlohu riadenia vozidla, a jedna z akcií nastavuje rýchlosť vozidla v intervale 0 - 130, tak actor bude generovať číslo z tohto intervalu. Critic ako vstup berie stavovú reprezentáciu, a akciu agenta, teda výstup actora. Výstupom critic siete je jedno číslo, a to očakávaná Q-hodnota danej akcie pri danom stave prostredia.

Algoritmus trénovania nájdete nižšie.

<img src="sources/lab08/ddpg.jpg" width="750">

Fungovanie `DDPG` agenta si ukážeme na prostredí [Pendulum](https://gym.openai.com/envs/Pendulum-v0/). [Stiahnite si ukážkovú implementáciu](sources/lab08/ddpg.py), a prediskutujte kód a jednotlivé kroky potrebné k dosiahnutiu riešenia.

**Úloha:** Na základe ukážkového riešenia skúste vytvoriť agenta s algoritmom DDPG, ktorý sa naučí navigovať v spojitom Gridworld prostredí cez spojité akcie. Ako akciu použijete uhol posunu z intervalu $<-\pi, \pi>$.